In [90]:
import tensorflow as tf
import pandas as pd
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
import re
import matplotlib.pyplot as plt
%matplotlib inline
import string
import random

In [91]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [92]:
labels=data_train.iloc[:, :-1].values
reviews = data_train.iloc[:, -1].values

In [93]:
def pre_processingdata(revierws):
  reviews_processed = []
  for review in reviews:
    review_good_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_good_one)
  word_reviews = []
  clean_reviews = []
  for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review)
  
  for statement in word_reviews:
    clean = []
    for w in statement.split():
      new_w = w.translate(str.maketrans('','','!#$%^&*<>?,./:;"[]{\}_-+='))
      if (new_w!=''):
        clean.append(new_w)
    clean_reviews.append(clean)
  return clean_reviews

In [94]:
data_train = pre_processingdata(reviews)

In [95]:
encoded_labels = []
for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    else:
        encoded_labels.append([0,0,1])
encoded_labels = np.array(encoded_labels)

In [96]:
EMBEDDING_DIM = 400 # HOW BIG IS EACH WORD VECTOR
MAX_VOCAB_SIZE = 10000 # HOW MANY UNIQUE WORDS TO USE
MAX_SEQUENCE_LENGTH = 300 # MAX NUMBER OF WORDS IN A COMMENT TO USE

In [97]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer  

In [98]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(data_train)
sequences_train = tokenizer.texts_to_sequences(data_train)
word_index = tokenizer.word_index

In [99]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [100]:
print("Shape of X_train and X validation tensor:", data.shape)
print("Shape of label train and validation tensor:", labels.shape)

Shape of X_train and X validation tensor: (1896, 300)
Shape of label train and validation tensor: (1896, 3)


In [101]:
labels_test=data_test.iloc[:, :-1].values
reviews_test = data_test.iloc[:, -1].values

In [102]:
encoded_labels_test = []
for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    else:
        encoded_labels_test.append([0,0,1])
encoded_labels_test = np.array(encoded_labels_test)

In [103]:
word_reviews_test = pre_processingdata(reviews_test)

In [104]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_text = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [105]:
print("Shape of X_train and X validation tensor:", data_test.shape)
print("Shape of label train and validation tensor:", labels_test.shape)

Shape of X_train and X validation tensor: (501, 2)
Shape of label train and validation tensor: (501, 3)


In [107]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load('Voca-vi.bin')
